In [19]:
import pandas as pd;
import numpy as np;
import os
import requests
import bs4 as bs
import nltk
import re
import time
import sys
import logging
import multiprocessing;
from wordcloud import WordCloud, ImageColorGenerator
from itertools import cycle;

from gensim.models import word2vec;


from nltk.corpus import words, stopwords
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize, sent_tokenize, pos_tag
#from gensim import corpora
import spacy
#spacy.load('en')
from spacy.lang.en import English

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KDTree;

import pyLDAvis.gensim
import warnings
import pickle
import gensim
from nltk.corpus import wordnet as wn
from gensim import corpora

import matplotlib.pyplot as plt

from gensim.models import Word2Vec 

warnings.filterwarnings('ignore')

In [49]:
folder_name = '/Users/michael/Downloads/SEC'
file_names = os.listdir(folder_name)
end_folder_name = '/Users/michael/Downloads/cleaned_SEC'

more_stop_words = ['company','months','products','operations','income','period','sales','market','business','stock','results',
                  'revenue','shares','share','revenues','assets','statements','value','costs','product','quarter','increase',
                  'expense','customers','performance','interest','charges','price','compensation','agreement','property','amount',
                  'markets','information','expenses','facility','customer','taxes','demand','payments',
                  'stringitemtype','textblockitemtype','monetaryitemtype','referencesreference','namespace','referencesno',
                  'definitionthe','definitionamount']

en_stop = nltk.corpus.stopwords.words('english')
en_stop.extend(more_stop_words)
parser = English()

In [26]:
def get_text_from_file(file):
    page_soup = bs.BeautifulSoup(file, 'lxml')
    divs = page_soup.find_all('div')
    all_text = ""
    for div in divs:
        text = div.getText()
        if len(text) > 200 and text[0] != '\xa0':
            ## removes all the /xa0 /n /t garbage and some extra spaces
            cleaner_text = re.sub('\s+', ' ', text).strip()
            all_text = all_text + " " + cleaner_text            
    return all_text

def get_all_files(files):
    all_files = []
    for file in files:
        text = get_text_from_file(file)
        #filing = combine_text(text)
        all_files.append(filing)
    return all_files

def tokenize(text):
    '''
    Separates each word from the others, making them lower case and removing URLs and possible screen names that show up.
    '''

    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

def combine_lists(list_of_lists):
    out = []
    for li in list_of_lists:
        out.append(li)
    return out

In [48]:
def clean_filing(start_folder,file_name,end_folder):
    
    ## get the raw file
    filing = open(start_folder + '/' + file_name).read()
    text = get_text_from_file(filing)
        
    if len(text)> 10**6 - 1 :
        #inc = 10**6
        #texts = [text[(i-1) * inc : i * inc]]
        text1 = text[:999999]
        text2 = text[999999:]
        tokens = tokenize(text1) + tokenize(text2)
        #tokens = combine_lists(texts)
    else:
        tokens = tokenize(text)
   
    new_tokens = []
    for tok in tokens:
        if len(tok) > 4 and len(tok) < 14 and tok[0] not in {'-', 'x'} and tok not in en_stop:
            new_tokens.append(tok)
            
    pos_tags = nltk.pos_tag(new_tokens)
    
    final_text = ""
    for pos_tag in pos_tags:
        if pos_tag[1][0] == "N":
            final_text = final_text + " " + pos_tag[0]

    full_file_name = end_folder + "/" + file_name
    text_file = open(full_file_name, "w+")
    text_file.write(final_text)
    text_file.close()

In [50]:
test = clean_filing(folder_name,file_names[0],end_folder_name)

' delaware jurisdiction i.r.s. organization hamlin hills executive offices jurisdiction i.r.s. organization hamlin hills executive offices registrant reports section securities exchange registrant reports filing requirements registrant interactive regulation registrant submit files filer filer reporting definitions filer exchange check growth check registrant transition accounting standards section exchange report management discussion analysis condition contain securities litigation reform management outlook expectations projections words effect estimates forecasts outlook plans project pursue target words expressions reference anticipate future position strategy measures strategy changes goals expansion growth plans success matters estimates accounting policies report assumptions analyses experience perception trends conditions developments factors circumstances guarantees forward place reliance forward risks uncertainties control differ forward risks uncertainties others truck produ